In [12]:
import os
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage.filters import gaussian, threshold_otsu
from skimage.transform import resize
from skimage import io
import pickle
from skimage.measure import regionprops

In [61]:
sourceDir='/data/xinyi/c2p/data/chromark'
segDir=os.path.join(sourceDir,'nuclear_masks')
imgDir=os.path.join(sourceDir,'raw_data')
conditions=['controls','headneck','meningioma', 'glioma']

In [317]:
outSize=128
updateImgs=False
savename='pathCentered_'+str(outSize)
for condition_i in conditions:
    print(condition_i)
    segDir_i=os.path.join(segDir,condition_i)
    imgDir_i=os.path.join(imgDir,condition_i)
    for stain in os.listdir(segDir_i):
#         if condition_i=='headneck' and stain=='dapi_gh2ax_lamin_cd3': #skipping incorrect images
#             continue
        print(stain)
        segDir_i_stain=os.path.join(segDir_i,stain)
        imgDir_i_stain=os.path.join(imgDir_i,stain)
        
        segPID2name={}
        for pID_dir in os.listdir(segDir_i_stain):
            pID=pID_dir.split('_')
            segPID2name[pID[0]]=pID_dir
        imgPID2name={}
        for pID_dir in os.listdir(imgDir_i_stain):
            pID=pID_dir.split('_')
            imgPID2name[pID[0]]=pID_dir
        for pID in segPID2name.keys():
            if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                continue
            print(pID)
            if pID not in imgPID2name:
                print('img not found '+pID)
                continue
            imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
            segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])
            if os.path.exists(os.path.join(imgDir_i_stain_p,savename+'_img')) and updateImgs==False:
                print('already processed')
                continue
            
            imgs_all=None
            imgNames=None
            for imgname in os.listdir(segDir_i_stain_p):
                if not os.path.exists(os.path.join(imgDir_i_stain_p,imgname)):
                    print('missing '+imgname)
                    continue
                seg=io.imread(os.path.join(segDir_i_stain_p,imgname))
                img=io.imread(os.path.join(imgDir_i_stain_p,imgname))
                #min-max
                img = img - np.min(img, axis=(0,1)).reshape(1,1,img.shape[2])
                img= img / np.max(img, axis=(0,1)).reshape(1,1,img.shape[2])
                cellstats=regionprops(seg)
                imgs_curr=np.zeros((len(cellstats),img.shape[2],outSize,outSize))
                for c in range(len(cellstats)):
                    centroid_c=cellstats[c].centroid
                    xstart=int(centroid_c[0]-outSize/2)
                    ystart=int(centroid_c[1]-outSize/2)
                    xend=min(xstart+outSize,img.shape[0])
                    yend=min(ystart+outSize,img.shape[1])

                    xstart_idx = 0
                    ystart_idx = 0
                    if xstart < 0:
                        xstart_idx = -xstart
                        xstart = 0
                    if ystart < 0:
                        ystart_idx = -ystart
                        ystart = 0
                    imgs_curr[c,:,xstart_idx:(xend-xstart+xstart_idx),ystart_idx:(yend-ystart+ystart_idx)]=np.moveaxis(img[xstart:xend,ystart:yend],2,0)
                if imgs_all is None:
                    imgs_all=imgs_curr
                    imgNames=np.repeat(imgname,len(cellstats))
                else:
                    imgs_all=np.concatenate((imgs_all,imgs_curr),axis=0)
                    imgNames=np.concatenate((imgNames,np.repeat(imgname,len(cellstats))))
            with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'wb') as output:
                pickle.dump(imgs_all,output,pickle.HIGHEST_PROTOCOL)
            with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'wb') as output:
                pickle.dump(imgNames,output,pickle.HIGHEST_PROTOCOL)



controls
dapi_gh2ax_lamin
HV5
already processed
HV4
already processed
HV2
already processed
HV10
already processed
HV1
already processed
HV7
already processed
HV9
already processed
HV8
already processed
HV3
already processed
HV6
already processed
dapi_cd8_cd4
HV1
already processed
HV8
already processed
HV6
already processed
HV7
already processed
HV4
already processed
HV5
already processed
HV3
already processed
HV2
already processed
headneck
dapi_gh2ax_lamin_cd3
P72
already processed
P63
already processed
P14
already processed
P44
already processed
P41
already processed
P56
already processed
P18
already processed
P55
already processed
P50
already processed
P24
already processed
dapi_cd16_cd4_cd8
P55
already processed
P56
already processed
P72
already processed
P14
already processed
P18
already processed
P41
already processed
P44
already processed
P63
already processed
P50
already processed
P24
already processed
meningioma
dapi_gh2ax_lamin_cd3
P27
already processed
P70
already processed
